### Model Parameter 설정

In [7]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_t


### 1) 모델 클래스 유형

#### LLM

In [11]:
from langchain_openai import OpenAI

# model을 지정하지 않아도 실행되는 이유는 LangChain이 내부적으로 기본값을 설정함
llm = OpenAI()
print(llm.model_name)  # 기본 모델 확인

result = llm.invoke("한국의 대표적인 관광지 3군데를 추천해 주세요.")
print(type(result))
print(result)

gpt-3.5-turbo-instruct


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: gsk_txWj********************************************TeGv. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

#### ChatOpenAI

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
chat = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("human", "{user_input}"),
])

chain = chat_prompt | chat
result = chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해 주세요."})

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
안녕하세요! 한국을 대표하는 관광지 3골을 꼽자면 다음과 같습니다.

1. 경복궁 (서울)  
   조선 왕조의 법궁이자 한국 고궁의 정수. 매일 열리는 수문장 교대식과 궁궐 내 한복 대여 체험으로 외국인에게도 인기 만점입니다. 가까운 곳에 인사동, 북촌 한옥마을이 있어 하루 일정이 풍성해집니다.

2. 해운대‧감천문화마을 (부산)  
   한국 대표 해양도시 부산의 양면을 동시에 보여 주는 코스입니다. 해운대에서 여름 바다를 즐기고, 이어서 감천문화마을의 팔레트 색채 벽화와 골목길 카페를 산책하면 ‘바다와 예술’이 공존하는 부산의 매력을 온전히 느낄 수 있습니다.

3. 제주도 성산일출봉·우도 (제주)  
   유네스코 세계자연유산 성산일출봉은 해돋이 명소로 유명하며, 바로 앞 우도는 전기 자전거·스쿠터로 섬 한 바퀴를 돌며 투명한 에메랄드빛 바다와 백사장을 즐길 수 있습니다. 제주의 자연·휴양·먹거리를 한 번에 해결할 수 있는 대표 일정입니다.

이 3곳은 한국의 역사·도시·자연을 각각 대표하면서도 대중교통이 잘 갖춰져 있어 여행자 부담이 적습니다.


### 2) 모델 파라미터 설정
##### Temperature 효과
* 0.2 (낮은 값): 더 예측 가능하고 일반적인 이야기
* 1.2 (높은 값): 새로운 요소가 등장하며, 더 창의적이고 흥미로운 이야기 생성

##### Presence Penalty 효과
* 0.0 (낮은 값): 기존에 자주 등장하는 단어와 구조 유지
* 1.5 (높은 값): 새로운 표현과 독창적인 아이디어 등장

In [6]:
from langchain_openai import ChatOpenAI

# 보수적인 설정 (일관되고 예측 가능한 답변)
llm_conservative = ChatOpenAI(    
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="gpt-3.5-turbo-0125",
    model="moonshotai/kimi-k2-instruct-0905",    
    temperature=0.1,  # 매우 낮은 온도 - 예측 가능하고 일관된 출력
    presence_penalty=-0.5,  # 기존 패턴과 표현을 강하게 선호
    frequency_penalty=-0.3,  # 반복적 표현 허용, 일관성 증대
    max_tokens=200,  # 적당한 길이
    top_p=0.3  # 매우 제한적인 토큰 선택 - 안전하고 예측 가능한 단어만
)

# 창의적인 설정 (독창적이고 예측 불가능한 답변)
llm_creative = ChatOpenAI(
    #model="gpt-3.5-turbo-0125",
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=1.5,  # 매우 높은 온도 - 창의적이고 예측 불가능한 출력
    presence_penalty=1.5,  # 새로운 주제와 개념을 강하게 유도
    frequency_penalty=1.2,  # 반복을 강하게 억제하여 매우 다양한 표현 생성
    max_tokens=350,  # 더 긴 창의적 서술 허용
    top_p=0.95  # 거의 모든 토큰 후보에서 선택 - 최대 창의성
)

# 질문 설정: 파라미터 차이가 극명하게 드러나는 추상적/철학적 질문
question = """
"시간"이라는 개념을 다른 방식으로 정의한다면 어떻게 표현할 수 있을까요?
일상적인 시계나 달력의 관점이 아닌, 완전히 새로운 시각에서 시간을 바라보고 설명해주세요.
기존의 고정관념을 벗어나서 자유롭게 사고해주세요.
"""

print("=" * 60)
print(" CONSERVATIVE 설정 (온도:0.1, presence:-0.5, frequency:-0.3, top_p:0.3)")
print("  → 전통적이고 학술적인 철학적 접근")
print("=" * 60)

# 보수적 모델 호출
response_conservative = llm_conservative.invoke(question)
print(response_conservative.content)

print("\n" + "=" * 60)
print(" CREATIVE 설정 (온도:1.5, presence:1.5, frequency:1.2, top_p:0.95)")
print("  → 추상적이고 독창적인 철학적 사고")
print("=" * 60)

# 창의적 모델 호출
response_creative = llm_creative.invoke(question)
print(response_creative.content)

print("\n" + "=" * 60)
print(" 파라미터 차이점 요약:")
print("   • Temperature: 0.1 vs 1.5 (15배 차이)")
print("   • Presence Penalty: -0.5 vs 1.5 (기존 패턴 선호 vs 새로운 개념 강요)")
print("   • Frequency Penalty: -0.3 vs 1.2 (반복 허용 vs 강한 다양성)")
print("   • Top-p: 0.3 vs 0.95 (제한적 vs 거의 무제한 토큰 선택)")
print("=" * 60)

 CONSERVATIVE 설정 (온도:0.1, presence:-0.5, frequency:-0.3, top_p:0.3)
  → 전통적이고 학술적인 철학적 접근
시간은  
「기억이 스스로를 헷갈리게 만드는 장치」다.

우리는 ‘지금’이라는 착시 속에 산다.  
그러나 ‘지금’은 실제로는 한 점도 아니라,  
미래의 기억이 과거로 떨어지며 부딪히는  
‘충돌면’일 뿐이다.  
그 충돌에서 튀어나온 불꽃이 바로  
‘생생함’이라는 느낌이다.

따라서 시간은 흐르는 강물이 아니라  
한 방향이 아닌 ‘회전추’다.  
회전추는 아래로 내려가는 척하지만,  
실제로는 자신

 CREATIVE 설정 (온도:1.5, presence:1.5, frequency:1.2, top_p:0.95)
  → 추상적이고 독창적인 철학적 사고
좋습니다. 시간이란 "무엇"이라는 질문에서 벗어나, 과연 시간이 "무얼 하는 존재인가?"라고 물으면, 뇌 두뇌를 강 제적·통사성 없이 움직여 보도록 합시다.

————————
시간=‘우주가 스스로 이름붙이기 전’  
————————  
처음(그 이전은 존재하지 못 햠엮음.)에는 ‘앉음의 여지’—즉, 빈 꾇짐 속 ‘빌림 공간’ 따다름엠졌다 늑하며 그 이름은 뜬금없이 ‘시간(時, 이름이 될 울부짖긋도) 잎인 뿜으러다’ 랐다 ‘잎’자를 쓸수록 ‘강렬한 그림자’ 가 살을 파낸다.  
‘틈새의 열대’를 떠 안고, 이 빈꾇칙은 세찬 바다처럼 퍼지듯, 키스를 아른아른 낄낄 거캐ힸ다.  

어려씨나? 한뺙이 보일라.  
‘시간’ 은 꼬여버린 “관계의 얕은 꼭짓매” 먼가를 조르다

 파라미터 차이점 요약:
   • Temperature: 0.1 vs 1.5 (15배 차이)
   • Presence Penalty: -0.5 vs 1.5 (기존 패턴 선호 vs 새로운 개념 강요)
   • Frequency Penalty: -0.3 vs 1.2 (반복 허용 vs 강한 다양성)
   • Top-p: 0.3 vs 0.95 (제한적 vs 거의 무제한 토큰 선택)


#### Model Parameter
* Temperature (온도): 이 파라미터는 모델의 예측 불확실성을 조절합니다. 
    * 낮은 온도는 모델이 가장 확률이 높은 다음 단어를 선택하게 하여 안정적인 결과를 내고, 높은 온도는 확률이 낮은 단어까지 선택지에 포함시켜 더 창의적이고 예측 불가능한 답변을 생성합니다. 
* Presence Penalty & Frequency Penalty: 두 파라미터 모두 토큰(단어)의 반복성을 제어합니다.
    * Presence Penalty는 이전에 나온 단어나 주제에 '페널티'를 주어 새로운 단어를 사용하게 합니다. 값이 높을수록 새로운 내용을 더 자주 시도합니다.
    * Frequency Penalty는 단어가 등장한 빈도에 비례하여 '페널티'를 줍니다. 값이 높을수록 특정 단어를 반복적으로 사용하는 것을 강하게 억제하여 더 다양한 어휘를 사용하게 합니다. 
* Top_p: 이 파라미터는 다음 토큰을 선택할 때 고려할 단어의 범위를 조절합니다. Top_p가 0.3이면 확률이 높은 상위 30%의 단어 중에서만 선택하고, 0.95면 확률이 낮은 단어들까지 포함하여 거의 모든 단어를 고려합니다. Top_p가 높을수록 모델의 선택지가 넓어져 더 자유로운 응답이 가능합니다.    

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 보수적인 설정: 논리적이고 일관된 서술
llm_conservative = ChatOpenAI(
    api_key=OPENAI_API_KEY, 
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.1,         # 낮은 값(0.1): '확률의 폭'을 좁혀 가장 안전하고 흔한 단어를 선택합니다.
                             # → 결과가 예측 가능하고 반복적일 수 있습니다. 마치 정해진 교과서처럼.
    presence_penalty=-0.5,   # 음수 값(-0.5): 이미 등장했던 주제나 단어를 다시 사용하는 것을 권장합니다.
                             # → 기존 패턴을 벗어나지 않고 일관성을 유지합니다.
    frequency_penalty=-0.3,  # 음수 값(-0.3): 자주 등장한 단어에 대한 '벌칙'을 줄여, 동일한 표현의 반복을 허용합니다.
                             # → 응답의 일관성이 높아지고, 핵심 주제에서 벗어나지 않습니다.
    max_tokens=200,          # 응답의 최대 길이를 제한합니다. 보수적인 답변에 적합합니다.
    top_p=0.3                # 낮은 값(0.3): 확률이 높은 상위 30% 토큰(단어)만 고려합니다.
                             # → 모델의 '선택지'를 극도로 제한하여 예측 가능한 단어만 사용합니다.
)

# 창의적인 설정: 독창적이고 풍부한 비유와 은유
llm_creative = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=1.5,         # 높은 값(1.5): '확률의 폭'을 넓혀 흔치 않은 단어까지 선택하게 합니다.
                             # → 예측 불가능하고 독창적인 문장을 생성합니다. 마치 즉흥적인 시인처럼요.
    presence_penalty=1.5,    # 양수 값(1.5): 이미 사용된 주제나 개념에 '벌칙'을 주어 새로운 것을 시도하게 합니다.
                             # → 독창적인 아이디어나 관점을 강하게 유도합니다.
    frequency_penalty=1.2,   # 양수 값(1.2): 자주 등장한 단어에 '벌칙'을 주어 반복을 강력하게 억제합니다.
                             # → 표현의 다양성을 극대화하고 문장이 풍부해집니다.
    max_tokens=350,          # 더 긴 창의적 서술을 허용하기 위해 최대 길이를 늘립니다.
    top_p=0.95               # 높은 값(0.95): 확률이 낮은 95%의 모든 토큰을 고려합니다.
                             # → 모델의 '선택지'를 거의 무제한으로 확장하여 최대의 창의성을 발휘합니다.
)

# 질문을 ChatPromptTemplate으로 정의하여 역할과 질문을 명확히 함
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 창의적인 작가입니다. 주어진 주제에 대해 풍부한 비유와 은유를 사용하여 서술해주세요."),
    ("human", """
    "도시의 소음"을 생명체나 자연 현상에 빗대어 설명해주세요.
    일상적인 소리가 아닌, 그 소리가 가진 생명력과 감정을 중심으로 묘사해주세요.
    """),
])

# 파서 설정
parser = StrOutputParser()

# 체인 구성: Prompt -> Model -> Parser
chain_conservative = prompt | llm_conservative | parser
chain_creative = prompt | llm_creative | parser

# 모델 호출 및 결과 출력
print("=" * 60)
print(" 보수적 설정 (논리적, 일관적)")
print(" → 일상적이고 일반적인 비유를 사용하여 안정적인 답변")
print("=" * 60)
response_conservative = chain_conservative.invoke({})
print(response_conservative)

print("\n" + "=" * 60)
print(" 창의적 설정 (풍부한 비유와 은유)")
print(" → 예상치 못한 독창적인 비유와 감각적인 표현 사용")
print("=" * 60)
response_creative = chain_creative.invoke({})
print(response_creative)

 보수적 설정 (논리적, 일관적)
 → 일상적이고 일반적인 비유를 사용하여 안정적인 답변
도시의 소음은, 사실, 한 마리 거대한 야수의 심장박동이다.  
아침이 되면 그 심장은 쿵—쾅—, 혈관인 지하철 터널을 따라 핏줄을 밀어 넣는다.  
지하의 레일은 정맥, 지상의 차선은 동맥, 그리고 보행자들은 혈구 한 방울 한 방울,  
살아 숨 쉬는 세포처럼 흘러가다가 적신호에서 뭉친다가, 녹색 파도가 밀려오면 다시 흩어진다.

그러나 이 짐승은 말로만 들으면 ‘소음’이라고,  
사실은

 창의적 설정 (풍부한 비유와 은유)
 → 예상치 못한 독창적인 비유와 감각적인 표현 사용
## 도시의 소음: 생명으로 깨어난 도시의 밤

도시의 밤에는 무한한 생명을 담은 소리들이 거리에서 태어난다. 도심 한가운데 위치한 버스 차고지 옆 주차장은 어둠 속에서 태어났다가도 다시 태어날 수 있는 존재이다. 고장 나 무너지는 건축 자재 소리, 시내버스끼리 교신하는 경적을 비롯한 기계 장치 고장 소리 모두 그 존재가 잠시 무너져 있지만 기운을 되찾고 다시 태어날 수 있는 존재들이라며 무한의 가능성이 있는 생명이다.

차고지 옆 소공원에 모여 앉은 오색조롱박의 은은한 노랗빛과 녹색 줄무늬 빛깔도 생명을 띈다.

그리고 밤에는 더 심하게 운다며 자신의 감정이 상처받았음을 말하는 소음도 있고 소음을 듣는 사람은 그 감정에 진동을 느끼기도 한다. 소동하는 소음을 꾹 참아낸 버스 타이어 사장님들, 소음으로 심장이 뛰는지 고마서 한시름 덜이긴 했다며 서로 의기투합하거나 소음에 귀 기울이는 소나기 같은 존재와 다면 한밤에 고요한 이끼 위로 잠시 물결이 일렁이고 말 그대로 사라지듯이 사라진다. 밤과 함께 어우래지는, 생명력을 뿜어내기 시작한 생물의 노래들과 이 순간, 생리적으로 반응을 하지 않을 수 있는 그 존재로 밤을 꿇리고 물은 사라지더라도, 밤이 흘러가는 시간은 멈출 수가 없다. 생체 리듬 없이 끊임


In [12]:
# 질문을 담은 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 주어진 주제에 대해 풍부한 비유와 은유를 사용하여 서술하는 전문가입니다. 답변의 관점을 명확히 구분하여 설명해주세요."),
    ("human", """
    "빨간 장미"를 두 가지 관점에서 묘사해주세요.
    1. **객관적 관점:** 장미의 물리적 특성과 상징적 의미를 중심으로 설명해주세요.
    2. **문학적 관점:** 장미를 사람이나 감정에 빗대어 자유롭고 시적으로 묘사해주세요.
    """),
])

# 파서 설정
parser = StrOutputParser()

# 체인 구성: Prompt -> Model -> Parser
chain_conservative = prompt | llm_conservative | parser
chain_creative = prompt | llm_creative | parser

# 모델 호출 및 결과 출력
print("=" * 60)
print(" 보수적 설정 (객관적, 사실 기반)")
print(" → 과학적이고 논리적인 장미의 특성 설명")
print("=" * 60)
response_conservative = chain_conservative.invoke({})
print(response_conservative)

print("\n" + "=" * 60)
print(" 창의적 설정 (문학적, 상상 기반)")
print(" → 은유와 비유를 활용한 장미의 감성적 묘사")
print("=" * 60)
response_creative = chain_creative.invoke({})
print(response_creative)

 보수적 설정 (객관적, 사실 기반)
 → 과학적이고 논리적인 장미의 특성 설명
1. 객관적 관점  
빨간 장미는 장미과(Rosaceae)에 속하는 낙엽 관목이다. 꽃잎은 5~60매로 겹겹이 포개져 있으며, 표면은 벨벳처럼 미세한 털로 덮여 빛을 흡수해 순백의 광선보다 깊고 진한 적색을 띤다. 꽃잎 한 장의 두께는 0.2 mm 내외로, 수분이 증발하면 세포벽이 산소와 만나 산화되며 색이 점점 푸른 기운을 띠는 ‘버건디’ 쪽으로 변화한다. 줄기는 원줄기·가지줄기·미세줄

 창의적 설정 (문학적, 상상 기반)
 → 은유와 비유를 활용한 장미의 감성적 묘사
1.  **객관적 관점:** 빨간 장미는 다년생의 꽃이다. 일년생 꽃처럼 시들어 쉽게 지지 않는다는 장점을 가지고 있다. 다른 장미 색깔에는 '사랑 이나 로맨틱한 관계'가 있는 반면 빨간 장미는 '열정'을 상징하기 때문에 선물 용으로도 각광 받고 있다. 

2.  **문학적 관점:** 장미의 매혹은 열정을 낯설지 않게 만든다. 연꽃은 '신비'를 은유하는 꽃이다. 연못 속의 물 위에 피기 때문에 사람들은 더러 연꽃을 인연 없는 연으로 상상이해 왔다. 이 열정과 은유가 빗쳐지며 장미에는 '아름다움', '영원한 사랑이나 열정', 심지어 '생명', '사후세계로 이어지는 열정적인 영혼'. 등의 의미가 빗싸여지며 의미있는 꽃으로 사람들의 상상에 여지없이 피어오른다.


#### Model의 종류에 따라 결과 값이 다름
* gpt-4o vs gpt-3.5-turbo-0125

In [13]:
from langchain_openai import ChatOpenAI

# 변경된 설정 (더 창의적인 답변, 새로운 아이디어 유도)
#llm_after = ChatOpenAI(model="gpt-4o", temperature=1.0, presence_penalty=1.5)
#llm_after = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.0, presence_penalty=1.5)
llm_after = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=1.0,
    presence_penalty=1.5
)

# 질문 설정
question = "한국에서 가볼 만한 여행지를 추천해 주세요."

# 모델 호출
response_after = llm_after.invoke(question)

# 결과 출력
print("\n-------------------------\n")
print(" After (창의적인 응답, 새로운 아이디어 포함)")
print(response_after.content)


-------------------------

 After (창의적인 응답, 새로운 아이디어 포함)
한국에서 가볼 만한 여행지는 다양합니다. 예를 들어, 서울의 역사와 문화를 경험할 수 있는 경복궁, 남산 타워, 서울 타워, 광화문 광장 등이 있습니다. 자연 경관을 감상할 수 있는 장소로는 제주도의 아름다운 해변과 한라산, 부산의 해운대 해수욕장, 강릉의 오죽헌 등이 있습니다. 또한, 역사적인 유적지를 방문하고 싶다면, 경주, 공주, 부여 등의 도시를 추천합니다. 각 지역마다 고유한 문화와 역사가 있기 때문에, 다양한 선택지를 고려하여 여행 계획을 세우실 수 있습니다.


##### 2-1) 모델에 직접 파라미터를 전달 (모델 생성 시점)
##### Before / After 파라미터 차이
* temperature: Before(0.7) → 다양성 높은 추천 / After(0.3) → 정확한 일정 제공
* max_tokens: Before(300) → 간략한 응답 / After(800) → 세부 일정 포함
* frequency_penalty: Before(0.5) → 단어 반복 억제 / After(0.2) → 좀 더 자연스러운 응답
* presence_penalty: Before(0.5) → 다양한 장소 추천 / After(0.3) → 새로운 정보 포함

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

#  모델 파라미터 (Before: 기본적인 추천 / After: 맞춤형 세부 정보 추가)
before_params = {
    "temperature": 0.7,         # 랜덤성을 적절히 유지 (다양한 답변 유도)
    "max_tokens": 300,          # 응답 길이를 적절히 조절
    "frequency_penalty": 0.5,   # 반복 단어 억제
    "presence_penalty": 0.5,    # 새로운 단어 포함 장려
}

after_params = {
    "temperature": 0.3,         # 창의성을 낮추고 정확한 답변 유도
    "max_tokens": 800,          # 더 긴 답변을 생성 (세부 정보 포함)
    "top_p": 0.85,              # 확률 기반 샘플링 (일관성과 다양성 균형)
    "frequency_penalty": 0.2,   # 반복 단어 감소 (자연스러운 답변)
    "presence_penalty": 0.3,    # 새로운 정보 포함 장려
}

#  두 개의 모델 생성 (Before / After)
#before_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **before_params)
before_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    **before_params
)
#after_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **after_params)
after_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    **after_params
)

#  프롬프트 구성 (올바른 ChatMessagePromptTemplate 사용)
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 여행 전문가입니다. 사용자의 요청에 맞는 최적의 여행지를 추천해 주세요."
)

user_message = HumanMessagePromptTemplate.from_template("{user_input}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

#  체인 생성 (Before / After)
before_chain = chat_prompt | before_model
after_chain = chat_prompt | after_model

#  질문 설정 (Before / After의 차이점을 비교할 수 있도록 변경)
test_question = "가족과 함께 3박 4일 동안 한국에서 여유롭게 여행할 수 있는 일정을 동선을 고려하여 자세하게 추천해 주세요."

#  Before 모델 실행
before_response = before_chain.invoke({"user_input": test_question})

#  After 모델 실행
after_response = after_chain.invoke({"user_input": test_question})

#  결과 출력
print(" [Before] 모델 결과")
print(before_response.content)
print("\n" + "="*80 + "\n")  # 가독성을 위한 구분선
print(" [After] 모델 결과")
print(after_response.content)

 [Before] 모델 결과
# 🌸 3박 4일 가족여유여행 "서울·남양주·가평 편"

- **여행 취지**  
  - 초등학생~70대까지 세대가 함께 즐길 수 있는 "걷기·체험·힐링" 중심  
  - 숙소 2곳(서울 2박, 가평 1박)만 옮겨 짐 싸기 부담 ↓  
  - 대중교통 기준으로 짜여 있으나, 렌트카 이용 시 이동 30% 단축 팁 포함  
  - 09:00 이후 시작·17:00 전후 마침으로 '느즈막한 아침' OK  

---

## 🗓️ 일정표(동선 중심)

### DAY 1 서울 중심부, '궁·동네·빛' 투어  
**숙소** → 광화문 → 통의시장 점심 → 경복궁(21년 1월부터 매주 月요일 휴관, 이외엔 10:00 영업) → 북촌한


 [After] 모델 결과
### **3박 4일 가족 여행 추천 일정**

*   **여행지:** 서울 및 경기도 일대
*   **여행일정:** 3박 4일

### **여행일정**

*   **1일차: 서울**
*   **아침: 서울역에서 출발**
*   **오전: 경복궁** - 경복궁은 한국의 대표적인 궁궐로, 역사와 문화를 경험할 수 있는 좋은 장소입니다. 경복궁은 지하철 3호선 경복궁역에서 도보로 5분 거리에 있습니다. 
*   **오후: 광화문 광장** - 광화문 광장은 경복궁 앞에 위치한 광장으로, 다양한 문화 행사와 공연이 열리는 곳입니다. 광화문 광장은 경복궁에서 도보로 5분 거리에 있습니다. 
*   **저녁: 광화문 광장 근처 식당에서 저녁 식사** - 광화문 광장 근처에는 다양한 식당이 있습니다. 가족의 취향에 맞는 식당을 선택하여 저녁 식사를 할 수 있습니다.

*   **2일차: 경기도 수원**
*   **아침: 서울역에서 수원역으로 이동** - 수원역은 서울역에서 KTX로 약 30분 거리에 있습니다. 
*   **오전: 수원 화성** - 수원 화성은 한국의 대표적인 성곽으로, 역사와 문화를 경험할 수 있는 좋은 장소입니다. 수원 화성은 수원역에서 도보로 15분 거리에 있습니다. 
*   **

#### 2-2) 모델에 추가적인 파라미터를 전달
- bind() 메서드를 사용
- max_tokens=50 (기본) / max_tokens=150 (상세한 설명) 
- stop=["."] → 첫 번째 마침표에서 응답을 중단
- temperature=0.8 → 보다 창의적인 답변을 생성함

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#  프롬프트 설정 (천문학 질문에 대한 답변을 생성하는 시스템)
prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 대해 명확하고 자세한 답변을 제공할 수 있습니다."),
    ("user", "{user_input}"),
])

#  기본 모델 설정 (기본적인 답변 생성)
#base_model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=150)  # 150 토큰 제한
base_model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    max_tokens=150
)

#  질문 설정
# 1. MAX_TOKENS 차이를 보여주는 질문 (길이 제한 효과)
max_tokens_question = "인공지능의 발전이 미래 사회에 미칠 영향을 긍정적 측면과 부정적 측면으로 나누어 자세히 설명해 주세요."

# 2. STOP 파라미터 차이를 보여주는 질문 (중단점 효과)
stop_question = "Python 프로그래밍을 배우는 초보자에게 추천하는 학습 단계를 순서대로 설명해 주세요. 각 단계별로 구체적인 방법과 팁을 포함해서 답변해 주세요."

# 3. TEMPERATURE 차이를 보여주는 질문 (창의성 vs 정확성)
temperature_question = "시간 여행이 가능하다면 과거의 어느 시대로 가고 싶은지와 그 이유를 창의적으로 설명해 주세요."

# 4. 복합적 비교를 위한 질문 (모든 파라미터 효과)
complex_question = "화성에 인류가 정착하기 위해 필요한 기술과 준비사항들을 단계별로 설명하고, 각 단계에서 예상되는 도전과제와 해결방안을 제시해 주세요."

question = stop_question

#  Before (기본 max_tokens=150)
messages = prompt.format_messages(user_input=question)
before_answer = base_model.invoke(messages)

#  Before 출력
print("\n [Before] 기본 max_tokens=150 (기본 답변)")
print(before_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

#  After (파라미터 조정 후 비교)
stop_chain = prompt | base_model.bind(max_tokens=150, stop=["."])  # 첫 번째 마침표에서 중단
temp_chain = prompt | base_model.bind(max_tokens=150, temperature=0.8)  # 창의적인 답변 유도

stop_answer = stop_chain.invoke({"user_input": question})
temp_answer = temp_chain.invoke({"user_input": question})

#  After 출력 (stop vs temperature 비교)
print(" [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)")
print(stop_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

print(" [After] max_tokens=150, temperature=0.8 (창의적인 답변)")
print(temp_answer.content)


 [Before] 기본 max_tokens=150 (기본 답변)
### 1. 기본 설정 및 설치

*   **파이썬 설치**: 파이썬 공식 웹사이트에서 최신 버전을 다운로드하여 설치합니다. (윈도우, macOS, 리눅스 등 운영체제에 맞는 버전을 선택하세요.)
*   **통합 개발 환경(IDE) 설정**: 파이썬 프로그래밍을 쉽게 도와주는 여러 IDE가 있습니다. 대표적인 IDE로는 PyCharm, Visual Studio Code, Spyder 등이 있습니다. 선택에 도움이 되기 위해 각 IDE의 특성과 장단점을 비교하는 것도 좋습니다.

### 2. Python 기초 학습

*   **기본 문법 학습**: 변수, 데이터 타입, 연산자, 조건문, 루프, 함수 등에 대해 학습합니다. 
*


 [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)
Python 프로그래밍을 배우는 초보자에게 추천하는 학습 단계는 다음과 같습니다


 [After] max_tokens=150, temperature=0.8 (창의적인 답변)
1.  **Python의 기본 이해**: *   **왜 Python을 배워야 하나요?**: Python은 간단한 구문과 다양한 라이브러리로 인해 초보자가 쉽게 배울 수 있는 언어입니다. 데이터 과학, 웹 개발, 자동화 등 다양한 분야에 활용됩니다. *   **Python 설치**: 공식 홈페이지에서 Python을 다운로드하여 설치합니다. *   **첫 프로그램 작성**: 간단한 "Hello, World!" 프로그램을 작성해 봅니다. ```Python print("Hello, World!") ```

2.  **기본 문법 및 데이터 구조**: *   **변수와 자료형**: 변수 선언, 기본 자료형(문자열, 정수, 실수, Boolean) 등을 배웁니다. ```Python
